# EOAP Generator

One of the Pathfinder delivery partners, Oxidian, has created a tool to help dev-ops specialists or specialist technicians to create compliant EOAPs that will run on the EODH. The eoap-gen tool can be found here: https://github.com/EO-DataHub/eoap-gen  

It describes itself as "a CLI tool for generating Earth Observation Application Packages including CWL workflows and Dockerfiles from user supplied python scripts".

## Requirements

There are three main requirements that are needed for the tool to create a working EOAP. These are:
* Python scripts. These must use [argparse](https://docs.python.org/3/library/argparse.html) or [click](https://click.palletsprojects.com/en/stable/) and the parameters will be mapped to the CWL `CommandLineTool` inputs
* A `pip` requirements file for each script being wrapped into the EOAP    __TO DO: check this can now also take conda__ 
* A compliant `eoap-gen` configuration file

## Steps

A full tutorial is provided with the repository (see https://github.com/EO-DataHub/eoap-gen/blob/main/ades_guide.md). Here, we will outline the main steps required in using the `eoap-gen` tool.

The first thing a user is required to do is understand the workflow that they want to wrap. At it's most simple the steps of a workflow are threefold: find your input data, process your input data, create a STAC output of the processed data. For the `eoap-gen` tool these steps will always be required.

When using the EODH it is recommended that the Python API client `pyeodh` is used to access the API endpooints on the Hub. To create the 

## Other tools

Other useful tools that you may want to try include:

### cwltool


### scriptcwl

https://github.com/NLeSC/scriptcwl and https://scriptcwl.readthedocs.io/en/latest/

### cwl-utils

https://github.com/common-workflow-language/cwl-utils and https://cwl-utils.readthedocs.io/en/latest/#


https://github.com/EO-DataHub/eoap-gen/blob/main/ades_guide.md